# Data collection activities
All the data collection activities are automated using user defined functions retrievable in the folder `scripts`.

In [6]:
from bs4 import BeautifulSoup
import requests as requests
from scripts import user, item, discussion, link, poll, bounty, comment
import csv
from tqdm import tqdm
import sqlite3

# Item scraping
The following code saves items data into a csv file, provided a range of item codes fixed by the operator.

First of all we need to initialize all the files for data collection

## Initialization of csv files

In [34]:
# Post items
file_path_post = "../data/post.csv"
row_head_post = ["Title",
                 "Category",
                 "Item code",
                 "Sats",
                 "Boost",
                 "Comments",
                 "Author",
                 "Tag",
                 "Timestamp",
                 "Main link",
                 "Body links",
                 "Sats received by comments",
                 "Comments item code",
                 ]
    
with open(file_path_post, 'w', encoding='utf_8_sig', newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_head_post)

In [35]:
# Comment items
file_path_comment = "../data/comment.csv"
row_head_comment = ["Item code",
                    "Sats",
                    "Boost",
                    "Comments",
                    "Author",
                    "Tag",
                    "Timestamp",
                    "Comments item code",
                    ]
    
with open(file_path_comment, 'w', encoding='utf_8_sig', newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_head_comment)

In [36]:
# Try to scrape 150 different items rather than in 'progressive item mode'
from random import sample

sampled_items = sample([*range(1,200000)], 10)


In [37]:
print(sampled_items)

[158222, 55308, 5707, 184266, 160623, 170054, 122519, 10545, 78662, 56820]


## Csv and data structure

### Columns to be used for all the post item scraping

- Title
- Category
- Item code
- Banner data
- Main link
- Body links
- Sats received by comments
- Comment item code

### Columns to be used for the comment item scraping
- Item code
- Banner data
- Comment item code

**NB**-> the `comment item code` in Comment item table could even be deleted, we can just keep it in order to eventually see the relationship between the comment and the comments to the specific comment

In [ ]:
# for i in tqdm(sampled_items):
#     try:
#         # Provided a string returns a bs4.BeautifulSoup object
#         url_posts = f'https://stacker.news/items/{i}'
#         response = requests.get(url_posts)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
#         
#         if item.detect_item_type(i, soup)=='comment':
#             entry = [str(i),
#                      comment.extract_banner(soup),
#                      comment.extract_comment_item_code(soup)
#                      ]
#             
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_comment, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
# 
#         elif item.detect_item_type(i, soup)=='link':
#             entry = [link.extract_title(soup),
#                      item.detect_item_type(i,soup),
#                      str(i),
#                      link.extract_banner(soup),
#                      link.extract_link(soup),
#                      link.extract_body_links(soup),
#                      link.extract_comment_stacked(soup),
#                      link.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='discussion':
#             entry = [discussion.extract_title(soup),
#                      item.detect_item_type(i,soup),
#                      str(i),
#                      discussion.extract_banner(soup),
#                      None,
#                      discussion.extract_body_links(soup),
#                      discussion.extract_comment_stacked(soup),
#                      discussion.extract_comment_item_code(soup)
#                      ]
#             
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#                         
#         elif item.detect_item_type(i, soup)=='poll':
#             entry = [poll.extract_title(soup),
#                      item.detect_item_type(i, soup),
#                      str(i),
#                      poll.extract_banner(soup),
#                      None,
#                      poll.extract_body_links(soup),
#                      poll.extract_comment_stacked(soup),
#                      poll.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='bounty':
#             entry = [bounty.extract_title(soup),
#                      item.detect_item_type(i, soup),
#                      str(i),
#                      bounty.extract_banner(soup),
#                      None,
#                      bounty.extract_body_links(soup),
#                      bounty.extract_comment_stacked(soup),
#                      bounty.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#     except:
#         continue


### Optimizing for the similarities between item post types

In the following chunck the functions applied are clustered for similarities, leading to three blocks:
- Comment
- Link
- Discussion/poll/bounty


In [38]:
for i in tqdm(sampled_items):
    try:
        # Provided a string returns a bs4.BeautifulSoup object
        url_posts = f'https://stacker.news/items/{i}'
        response = requests.get(url_posts)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if item.detect_item_type(i, soup)=='comment':
            entry = [str(i),
                     comment.extract_banner(soup)['sats'],
                     comment.extract_banner(soup)['boost'],
                     comment.extract_banner(soup)['comments'],
                     comment.extract_banner(soup)['author'],
                     comment.extract_banner(soup)['tag'],
                     comment.extract_banner(soup)['timestamp'],
                     comment.extract_comment_item_code(soup)
                     ]
            
            # Appends every new profile to a csv file in the provided path
            try:
                with open(file_path_comment, 'a', encoding='utf_8_sig', newline="") as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow(entry)
            except:
                print('Error while processing data')

        elif item.detect_item_type(i, soup)=='link':
            entry = [link.extract_title(soup),
                     item.detect_item_type(i,soup),
                     str(i),
                     link.extract_banner(soup)['sats'],
                     link.extract_banner(soup)['boost'],
                     link.extract_banner(soup)['comments'],
                     link.extract_banner(soup)['author'],
                     link.extract_banner(soup)['tag'],
                     link.extract_banner(soup)['timestamp'],
                     link.extract_link(soup),
                     link.extract_body_links(soup),
                     link.extract_comment_stacked(soup),
                     link.extract_comment_item_code(soup)
                     ]
            # Appends every new profile to a csv file in the provided path
            try:
                with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow(entry)
            except:
                print('Error while processing data')
        
        elif item.detect_item_type(i, soup) in ['discussion', 'poll', 'bounty']:
            entry = [discussion.extract_title(soup),
                     item.detect_item_type(i,soup),
                     str(i),
                     discussion.extract_banner(soup)['sats'],
                     discussion.extract_banner(soup)['boost'],
                     discussion.extract_banner(soup)['comments'],
                     discussion.extract_banner(soup)['author'],
                     discussion.extract_banner(soup)['tag'],
                     discussion.extract_banner(soup)['timestamp'],
                     None,
                     discussion.extract_body_links(soup),
                     discussion.extract_comment_stacked(soup),
                     discussion.extract_comment_item_code(soup)
                     ]
            
            # Appends every new profile to a csv file in the provided path
            try:
                with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow(entry)
            except:
                print('Error while processing data')
        
    except:
        continue


100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


--------------------------------------------------------------------------------
## Saving with SQLite 

In [7]:
# Try to scrape 150 different items rather than in 'progressive item mode'
from random import sample

sampled_items = sample([*range(1,250000)], 5000)

In [8]:
conn = sqlite3.connect('../data/stacker_news.sqlite')
cur = conn.cursor()

sql_comment = """
DROP TABLE IF EXISTS comments;
CREATE TABLE comments (
    ItemCode TEXT,
    Sats TEXT,
    Boost TEXT,
    Comments TEXT,
    Author TEXT,
    Tag TEXT,
    Timestamp TEXT,
    CommentsItemCode TEXT,
    PRIMARY KEY (ItemCode))
"""

sql_post = """
DROP TABLE IF EXISTS post;
CREATE TABLE post (
    Title TEXT,
    Category TEXT,
    ItemCode TEXT,
    Sats TEXT,
    Boost TEXT,
    Comments TEXT,
    Author TEXT,
    Tag TEXT,
    Timestamp TEXT,
    MainLink TEXT,
    BodyLinks TEXT,
    SatsReceivedComments TEXT,
    CommentsItemCode TEXT,
    PRIMARY KEY (ItemCode))
"""

cur.executescript(sql_comment)
cur.executescript(sql_post)

conn.commit()
conn.close()

In [9]:
insert_comment = """
INSERT INTO comments (
    ItemCode,
    Sats,
    Boost,
    Comments,
    Author,
    Tag,
    Timestamp,
    CommentsItemCode
    ) values (?, ?, ?, ?, ?, ?, ?, ?)
"""

insert_post = """
INSERT INTO post (
    Title,
    Category,
    ItemCode,
    Sats,
    Boost,
    Comments,
    Author,
    Tag,
    Timestamp,
    MainLink,
    BodyLinks,
    SatsReceivedComments,
    CommentsItemCode
    ) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

In [37]:
# conn = sqlite3.connect('../data/stacker_news.sqlite')
# cur = conn.cursor()
# 
# for i in tqdm(sampled_items):
#     try:
#         # Provided a string returns a bs4.BeautifulSoup object
#         url_posts = f'https://stacker.news/items/{i}'
#         response = requests.get(url_posts)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
#         
#         if item.detect_item_type(i, soup)=='comment':
#             # Insert every new entry into a new row in the provided DB
#             entry = (str(i),
#                      str(comment.extract_banner(soup)['sats']),
#                      str(comment.extract_banner(soup)['boost']),
#                      str(comment.extract_banner(soup)['comments']),
#                      str(comment.extract_banner(soup)['author']),
#                      str(comment.extract_banner(soup)['tag']),
#                      str(comment.extract_banner(soup)['timestamp']),
#                      str(comment.extract_comment_item_code(soup))
#                      )
#             try:
#                 cur.execute(insert_comment, entry)
#             except:
#                 print('Error while processing item ', i)
#     except:
#         print(i)
#                 
# conn.commit()
# cur.close()
# conn.close()

 32%|███▎      | 13/40 [00:09<00:18,  1.49it/s]

77162


100%|██████████| 40/40 [00:30<00:00,  1.30it/s]


Note that every entry in the sql statement is transformed into a string, only the `None` values (`NULL` in SQL) are left as is.

In [10]:
conn = sqlite3.connect('../data/stacker_news.sqlite')
cur = conn.cursor()

for i in tqdm(sampled_items):
    try:
        # Provided a string returns a bs4.BeautifulSoup object
        url_posts = f'https://stacker.news/items/{i}'
        response = requests.get(url_posts)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if item.detect_item_type(i, soup)=='comment':
            # Insert every new entry into a new row in the provided DB
            entry = (str(i),
                     str(comment.extract_banner(soup)['sats']),
                     str(comment.extract_banner(soup)['boost']),
                     str(comment.extract_banner(soup)['comments']),
                     str(comment.extract_banner(soup)['author']),
                     str(comment.extract_banner(soup)['tag']),
                     str(comment.extract_banner(soup)['timestamp']),
                     str(comment.extract_comment_item_code(soup))
                     )
            try:
                cur.execute(insert_comment, entry)
            except:
                print('Error while processing item ', i)

        elif item.detect_item_type(i, soup)=='link':
            # Appends every new profile to a csv file in the provided path
            entry = (str(link.extract_title(soup)),
                     str(item.detect_item_type(i,soup)),
                     str(i),
                     str(link.extract_banner(soup)['sats']),
                     str(link.extract_banner(soup)['boost']),
                     str(link.extract_banner(soup)['comments']),
                     str(link.extract_banner(soup)['author']),
                     str(link.extract_banner(soup)['tag']),
                     str(link.extract_banner(soup)['timestamp']),
                     str(link.extract_link(soup)),
                     str(link.extract_body_links(soup)),
                     str(link.extract_comment_stacked(soup)),
                     str(link.extract_comment_item_code(soup))
                     )
            try:
                cur.execute(insert_post, entry)
            except:
                print('Error while processing item ', i)
        
        elif item.detect_item_type(i, soup) in ['discussion', 'poll', 'bounty']:
            entry = (str(discussion.extract_title(soup)),
                     str(item.detect_item_type(i,soup)),
                     str(i),
                     str(discussion.extract_banner(soup)['sats']),
                     str(discussion.extract_banner(soup)['boost']),
                     str(discussion.extract_banner(soup)['comments']),
                     str(discussion.extract_banner(soup)['author']),
                     str(discussion.extract_banner(soup)['tag']),
                     str(discussion.extract_banner(soup)['timestamp']),
                     None,
                     str(discussion.extract_body_links(soup)),
                     str(discussion.extract_comment_stacked(soup)),
                     str(discussion.extract_comment_item_code(soup))
                     )
            
            # Appends every new profile to a csv file in the provided path
            try:
                cur.execute(insert_post, entry)
            except:
                print('Error while processing item ', i)
        
    except:
        continue

conn.commit()
cur.close()
conn.close()

100%|██████████| 5000/5000 [59:01<00:00,  1.41it/s]  
